In [33]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

In [34]:
# Завантаження даних з CSV файлу
df = pd.read_csv('../data/internet_service_churn.csv')

# Перегляд перших кількох рядків
df.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


In [35]:
# Перегляд відсутніх значень
df.isnull().sum()

id                                 0
is_tv_subscriber                   0
is_movie_package_subscriber        0
subscription_age                   0
bill_avg                           0
reamining_contract             21572
service_failure_count              0
download_avg                     381
upload_avg                       381
download_over_limit                0
churn                              0
dtype: int64

#Обробка відсутніх значень: Заповнення або видалення.

In [36]:
# Заповнення відсутніх значень медіанними значеннями для download_avg та upload_avg
#df['download_avg'] = df['download_avg'].fillna(df['download_avg'].median())
#df['upload_avg'] = df['upload_avg'].fillna(df['upload_avg'].median())

# Заповнення відсутніх значень середнім значенням значеннями для download_avg та upload_avg 
#df['download_avg'] = df['download_avg'].fillna(df['download_avg'].mean())
#df['upload_avg'] = df['upload_avg'].fillna(df['upload_avg'].mean())

#Видалення рядків, де є пропущені значення в стовпцях download_avg та upload_avg
df = df.dropna(subset=['download_avg', 'upload_avg'])

# Видалення стовпця reamining_contract, якщо його заповнення не має сенсу
#df.drop(columns=['reamining_contract'], inplace=True)

#Заповнення відсутніх значень для reamining_contract(залишок по контракту) на 0
df['reamining_contract'] = df['reamining_contract'].fillna(0)

# Видалення стовпця 'id'
df = df.drop('id', axis=1)

In [37]:
#Перевіряємо які є унікальні значення в download_over_limit
unique_values = df['download_over_limit'].value_counts()

print(unique_values)

download_over_limit
0    67992
1      766
6      688
2      560
7      504
3      498
4      456
5      429
Name: count, dtype: int64


In [38]:
# Замінюємо всі значення в download_over_limit, не рівні 0, на 1
df['download_over_limit'] = df['download_over_limit'].apply(lambda x: 1 if x != 0 else 0)

# Перевіряємо значення в download_over_limit
print(df['download_over_limit'].value_counts())

download_over_limit
0    67992
1     3901
Name: count, dtype: int64


In [39]:
df.head()

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,0,0,8.22,0,0.00,0,0.0,0.0,0,1
2,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,0,0,6.87,21,0.00,1,0.0,0.0,0,1
4,0,0,6.39,0,0.00,0,0.0,0.0,0,1


In [40]:
# Перегляд відсутніх значень для переконання що все заповнене
df.isnull().sum()

is_tv_subscriber               0
is_movie_package_subscriber    0
subscription_age               0
bill_avg                       0
reamining_contract             0
service_failure_count          0
download_avg                   0
upload_avg                     0
download_over_limit            0
churn                          0
dtype: int64

Кодування категоріальних змінних:
У наших даних є кілька категоріальних змінних:
is_tv_subscriber і is_movie_package_subscriber є бінарними змінними (0 або 1), тому вони не потребують додаткового кодування.
Для цього кроку достатньо залишити ці змінні як є, оскільки вони вже кодуються в бінарному вигляді.

Стандартизація

In [41]:
# Вибір числових ознак для стандартизації
numerical_columns = ['subscription_age', 'bill_avg', 'reamining_contract', 
                     'service_failure_count', 'download_avg', 'upload_avg']

df_standart_norm = df.copy()
#LightGBM не потребує нормалізації, а Logistic Regression так, тому використовуєм StandardScaler
scaler = StandardScaler()
df_standart_norm[numerical_columns] = scaler.fit_transform(df_standart_norm[numerical_columns])

#як варіант якщо потрібно для інших моделей
#df_min_max_norm = df
#scaler = MinMaxScaler()
#df_min_max_norm[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Перегляд результату
df_standart_norm.head()

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,1,0,4.684387,0.453666,-0.537157,-0.336792,-0.556575,-0.192699,0,0
1,0,0,2.844086,-1.441932,-0.747446,-0.336792,-0.689055,-0.426943,0,1
2,1,0,3.184517,-0.228749,-0.747446,-0.336792,-0.472986,-0.335282,0,1
3,0,0,2.178025,0.150371,-0.747446,0.884914,-0.689055,-0.426943,0,1
4,0,0,1.941204,-1.441932,-0.747446,-0.336792,-0.689055,-0.426943,0,1


In [42]:
df.head()

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,0,0,8.22,0,0.00,0,0.0,0.0,0,1
2,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,0,0,6.87,21,0.00,1,0.0,0.0,0,1
4,0,0,6.39,0,0.00,0,0.0,0.0,0,1


In [43]:
df.to_csv('../data/data_filled_notNormalized.csv', index=False)
df_standart_norm.to_csv('../data/data_normalized.csv', index=False)

In [44]:
# Сохранение scaler в файл
joblib.dump(scaler, '../data/scaler.pkl')

['../data/scaler.pkl']